In [1]:
import matplotlib.pyplot as plt
import matplotlib.table as tbl
import math
import numpy as np
import pandas as pd

import csv
import os

In [2]:
# Load data

# Expects all relevant files to be in this folder
folder = 'FDs_ManySizeBy1'
files = [f for f in os.listdir(folder) if 'Lung' in f]

df_list = []
for f in files:
    df_dict = {}
    with open(os.path.join(folder, f), 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        csv_data = list(csv_reader)
    
    sc_arr = np.asarray(csv_data[4:]).astype(int)
    sizes, counts = sc_arr[:, 0], sc_arr[:, 1]
    
    df_dict = {
        'SID':                 '_'.join(f.split('_')[:-2]),
        'Lobe':                '',
        'Fractal_Dimension':   float(csv_data[0][1]),
        'Coefficients':        np.array([csv_data[1][1], csv_data[2][1]]).astype(float),
        'Sizes':               sizes,
        'Counts':              counts,
    }
    
    if 'left' in f:
        df_dict['Lobe'] = 'left'
    elif 'right' in f:
        df_dict['Lobe'] = 'right'
    else:
        df_dict['Lobe'] = 'whole'
    
    df_list.append(df_dict)

df = pd.DataFrame(df_list)
display(df)

,SID,Lobe,Fractal_Dimension,Coefficients,Sizes,Counts
0,001_pect_phfirst_060413,left,2.364190,"[-2.364189983125097, 14.019521717347565]","[102, 101, 100, 99, 98, 97, 96, 95, 94, 93, 92...","[20, 21, 21, 23, 23, 25, 26, 27, 28, 28, 28, 2..."
1,001_pect_phfirst_060413,right,2.438564,"[-2.4385639415290665, 14.549878183268873]","[117, 116, 115, 114, 113, 112, 111, 110, 109, ...","[21, 22, 22, 22, 22, 22, 23, 23, 23, 24, 24, 2..."
2,001_pect_phfirst_060413,whole,2.391798,"[-2.3917982846549086, 14.971253130452506]","[141, 140, 139, 138, 137, 136, 135, 134, 133, ...","[21, 21, 22, 22, 22, 24, 25, 26, 26, 26, 26, 2..."
3,002_pect_phfirst_050913,left,2.366780,"[-2.3667796740728493, 13.852465434476752]","[72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 6...","[42, 43, 45, 46, 50, 49, 51, 53, 55, 58, 60, 6..."
4,002_pect_phfirst_050913,right,2.360379,"[-2.3603791734279502, 14.099230785534516]","[100, 99, 98, 97, 96, 95, 94, 93, 92, 91, 90, ...","[20, 23, 25, 26, 28, 31, 34, 34, 36, 36, 38, 3..."
...,...,...,...,...,...,...
671,387_pect_phfirst_090117,right,2.358504,"[-2.358503572008447, 13.87765616590567]","[97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 87, 8...","[22, 23, 24, 26, 27, 27, 29, 29, 29, 29, 30, 3..."
672,387_pect_phfirst_090117,whole,2.383237,"[-2.383236511704936, 14.626852384443659]","[125, 124, 123, 122, 121, 120, 119, 118, 117, ...","[21, 21, 22, 22, 23, 23, 24, 24, 26, 27, 28, 2..."
673,388_pect_phfirst_080516,left,2.292465,"[-2.292464889522172, 12.583582675415805]","[61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 5...","[22, 24, 26, 28, 29, 30, 30, 31, 32, 33, 34, 3..."
674,388_pect_phfirst_080516,right,2.358191,"[-2.3581908945640406, 13.013902640245513]","[69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 59, 5...","[19, 20, 20, 21, 22, 24, 26, 27, 27, 30, 33, 3..."


In [24]:
# Compute FD!

# [-(box_sizes_ind-1):]
box_sizes_ind = 20

folder = 'FDs_20test'
os.makedirs(folder, exist_ok=True)

for num in range(0, len(df)):
    sid = df['SID'][num]
    lobe = df['Lobe'][num]
    sizes = df['Sizes'][num][-(box_sizes_ind-1):]
    counts = df['Counts'][num][-(box_sizes_ind-1):]
    sizes_log = np.log(sizes)
    counts_log = np.log(counts)
    
    poly = np.polynomial.polynomial.Polynomial.fit(x=sizes_log, y=counts_log, deg=1, domain=[])
    
    cells = []
    cells.append(['Fractal_Dimension', f'{-poly.coef[1]:.16f}'])
    cells.append(['Coefficients', f'{poly.coef[1]:.16f}'])
    cells.append(['', f'{poly.coef[0]:.16f}'])
    cells.append(['Size', f'Box_Count'])
    
    for size, count in zip(sizes, counts):
        cells.append([f'{size}',f'{count}'])
        
    f = open(f'{folder}/{sid}_{lobe}LungVesselParticles_fd.csv', 'w')
    writer = csv.writer(f, delimiter=',', lineterminator='\n')
    for x in cells:
        writer.writerow(x)

    f.close()